In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "data/scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [5]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    # Remove 'header' and 'reserves' line in html
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [6]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score

In [7]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [8]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [9]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)

    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)
    game["home"] = [0,1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 9191
200 / 9191
300 / 9191
400 / 9191
500 / 9191
600 / 9191
700 / 9191
800 / 9191
900 / 9191
1000 / 9191
1100 / 9191
1200 / 9191
1300 / 9191
1400 / 9191
1500 / 9191
1600 / 9191
1700 / 9191
1800 / 9191
1900 / 9191
2000 / 9191
2100 / 9191
2200 / 9191
2300 / 9191
2400 / 9191
2500 / 9191
2600 / 9191
2700 / 9191
2800 / 9191
2900 / 9191
3000 / 9191
3100 / 9191
3200 / 9191
3300 / 9191
3400 / 9191
3500 / 9191
3600 / 9191
3700 / 9191
3800 / 9191
3900 / 9191
4000 / 9191
4100 / 9191
4200 / 9191
4300 / 9191
4400 / 9191
4500 / 9191
4600 / 9191
4700 / 9191
4800 / 9191
4900 / 9191
5000 / 9191
5100 / 9191
5200 / 9191
5300 / 9191
5400 / 9191
5500 / 9191
5600 / 9191
5700 / 9191
5800 / 9191
5900 / 9191
6000 / 9191
6100 / 9191
6200 / 9191
6300 / 9191
6400 / 9191
6500 / 9191
6600 / 9191
6700 / 9191
6800 / 9191
6900 / 9191
7000 / 9191
7100 / 9191
7200 / 9191
7300 / 9191
7400 / 9191
7500 / 9191
7600 / 9191
7700 / 9191
7800 / 9191
7900 / 9191
8000 / 9191
8100 / 9191
8200 / 9191
8300 / 9191
8400 / 9191
8

In [10]:
games_df = pd.concat(games, ignore_index=True)

In [11]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18377,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
18378,240.0,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,...,25.0,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True
18379,240.0,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,...,25.6,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False
18380,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [12]:
games_df.to_csv("nba_games_2023.csv")